In [1]:
import os
import torch
import numpy as np
import copy as cp
import math
import pickle
import json
import multi_loss_model as mlmodel

### LOAD DATA

In [2]:
labels_names=os.listdir('../../../pedestrianDepth-baseline/MonoDepth-PyTorch/data/pif_all/')
labels_names.sort()

In [3]:
val_labels=open("../../intermediate-data/kitti-pifpaf/val.txt", "r").read().split('\n')

In [4]:
pifpafbox = []
for i, label in enumerate(labels_names):
    if label[0:6] in val_labels:
        with open('../../../pedestrianDepth-baseline/MonoDepth-PyTorch/data/pif_all/'+label) as inputfile:
            temp = json.load(inputfile)

            if temp != []:
                for person in temp:
                    new = {}
                    new['ppbox'] = [x/2 for x in person['bbox']]

                    keyp = [x/2 for x in person['keypoints']]
                    
                    
                    #confidence = float(np.sort(np.array(keyp[2::3]))[-3])/2
                    confidence = sum(keyp[2::3])/(2*len(keyp[2::3]))
                    
                    #print(confidence, sum(keyp[2::3])/(2*len(keyp[2::3])))
                    
                    new['confidence'] = confidence

                    del keyp[2::3]

                    new['image_id'] = label[0:6]

                    new['keypoints'] = np.array(keyp).reshape(17,2)
                    
                    if new['confidence']>0.5:
                        #new['confidence']=1.0
                        pifpafbox.append(new)

print('we have',len(pifpafbox),'pifpaf pedestrians')

we have 1980 pifpaf pedestrians


### TRANSFORM DATA

In [5]:
factor=0.1
for person in pifpafbox:
    person['ppbox'][0]=person['ppbox'][0]-factor*(person['ppbox'][2]-person['ppbox'][0])
    person['ppbox'][1]=person['ppbox'][1]-factor*(person['ppbox'][3]-person['ppbox'][1])
    person['ppbox'][2]=person['ppbox'][2]+factor*(person['ppbox'][2]-person['ppbox'][0])
    person['ppbox'][3]=person['ppbox'][3]+factor*(person['ppbox'][3]-person['ppbox'][1])

In [6]:
inputs = np.array([x['keypoints'].flatten() for x in pifpafbox])


In [7]:
def infer(inputs, model):
    
    def convert_sin2angle(data):
        angles=[]
        for angle in data:
            angles.append(math.atan2(angle[0],angle[1])*180/math.pi)
        return np.array(angles)
    
    def fix_orientation(angle):
        if angle<0:
            return 360+angle
        if angle>360:
            return angle-360
        else:
            return angle
    
    inputs = torch.from_numpy(inputs).float().to(device)
    outputs = model(inputs)
    [orientations, dimensions, positions] = (outputs[:,:2], outputs[:,2:5],
                                             outputs[:,5:8])
    [orientations, dimensions, positions] = (orientations.cpu().detach().numpy(),
                                                      dimensions.cpu().detach().numpy(),
                                                      positions.cpu().detach().numpy())
    orientations = convert_sin2angle(orientations)
    orientations = np.array(list(map(fix_orientation, orientations)))
    
    return orientations, dimensions, positions

### LOAD MODEL

In [8]:
if torch.cuda.device_count()>1:
    #torch.cuda.device(1)
    torch.cuda.set_device(0)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#device="cpu"

In [9]:
with torch.no_grad():
    net = mlmodel.LinearModel(34, 8)
    net=net.to(device)
    net.eval()
    params = torch.load('../../intermediate-data/kitti-pifpaf/net-kitti-multi.pkl')
    net.load_state_dict(params)

In [10]:
out_sample = infer(inputs, net)


In [11]:
out_sample[0]

array([194.19929364, 111.41248195,  24.45119905, ..., 170.29289454,
       298.72491544, 212.56430457])

In [12]:
for i, detection in enumerate(pifpafbox):
    detection['pred_orient'] = np.round(out_sample[0][i],2)
    detection['pred_dim'] = np.round(out_sample[1][i],2)
    detection['pred_pos'] = np.round(out_sample[2][i],2)
    detection['obs_angle'] = math.atan2(detection['pred_pos'][2],detection['pred_pos'][0])*180/math.pi
    detection['truncation'] = 0.00
    detection['occlusion'] = 0

In [13]:
pifpafbox[13]

{'ppbox': [926.4931039333344,
  216.3945704460144,
  994.3396781492233,
  336.3778736972809],
 'confidence': 0.5867647058823531,
 'image_id': '000065',
 'keypoints': array([[963.1 , 229.45],
        [967.  , 227.25],
        [959.8 , 226.3 ],
        [971.3 , 230.55],
        [953.5 , 228.  ],
        [980.2 , 246.5 ],
        [942.25, 245.5 ],
        [988.15, 271.35],
        [936.05, 269.  ],
        [971.9 , 269.2 ],
        [946.3 , 274.95],
        [971.6 , 290.25],
        [947.5 , 285.75],
        [970.65, 308.2 ],
        [932.1 , 293.4 ],
        [980.5 , 325.45],
        [939.75, 324.75]]),
 'pred_orient': 160.1,
 'pred_dim': array([1.23, 0.47, 0.6 ], dtype=float32),
 'pred_pos': array([3.29, 1.53, 6.16], dtype=float32),
 'obs_angle': 61.89372986548645,
 'truncation': 0.0,
 'occlusion': 0}

### PRINT RESULTS

In [14]:
for label in val_labels:
    #print(label)
    pedestrians = [d for d in pifpafbox if d['image_id'] in [label]]
    
    text_file = open('../../../eval_kitti/build/results/benchmark-kitti/data/'+label+".txt", "w")
    
    for pedestrian in pedestrians:
        obs_angle = (pedestrian['obs_angle']-180)*math.pi/180
        
        bbox = [min(pedestrian['keypoints'][:,0]), min(pedestrian['keypoints'][:,1]),
                max(pedestrian['keypoints'][:,0]), max(pedestrian['keypoints'][:,1])]
        
        r_angle = math.atan2(pedestrian['pred_pos'][0] , pedestrian['pred_pos'][2])*180/math.pi
        
        abs_orient = pedestrian['pred_orient']+r_angle
        
        abs_orient -= 90
        
        #abs_orient = (abs_orient if abs_orient<360 else abs_orient-360)
        
        if abs_orient>180:
            abs_orient = abs_orient-360
        if abs_orient<-180:
            abs_orient = abs_orient+360
            
        
        abs_orient = abs_orient*math.pi/180
        
        
        
        
        #print(pedestrian)
        to_write = ("Pedestrian "+str(pedestrian['truncation'])+" "+str(pedestrian['occlusion'])+" "+str(round(obs_angle,2))+" "+str(bbox[0])
                    +' '+str(bbox[1])+' '+str(bbox[2])+' '+str(bbox[3])+' '+str(round(pedestrian['pred_dim'][0],2))+' '+str(round(pedestrian['pred_dim'][1],2))+' '
                    +str(round(pedestrian['pred_dim'][2],2))+' '+str(round(pedestrian['pred_pos'][0],2))+' '+
                    str(round(pedestrian['pred_pos'][1],2))+' '+str(round(pedestrian['pred_pos'][2],2))+' '+
                    str(round(abs_orient,2))+' '+str(round(pedestrian['confidence'],2))+'\n')
        text_file.write(to_write)
    text_file.close()

### COMPUTE ERROR

In [58]:
labels_names=os.listdir('../../../pedestrianDepth-baseline/MonoDepth-PyTorch/data/training/label_2/')
labels_names.sort()

In [59]:
gt = []

for i, label in enumerate(labels_names):
    if label[0:6] in val_labels:
        temp=[]
        with open('../../../pedestrianDepth-baseline/MonoDepth-PyTorch/data/training/label_2/'+label) as inputfile:
            for line in inputfile:
                temp.append(line.strip().split(' '))
        for line in temp:
            if line[0]=='Pedestrian':
                
                pedestrian = {}
                
                pedestrian['image_id'] = label[:6]
                
                pedestrian['truncation'] = float(line[1])
                
                pedestrian['occlusion'] = int(line[2])
                
                pedestrian['obs_angle'] = float(line[3])*180/math.pi + 180
                
                #pedestrian['obs_angle'] = (pedestrian['obs_angle'] if pedestrian['obs_angle']<360 else pedestrian['obs_angle']-360)
                
                pedestrian['gt_bbox'] = [float(i) for i in line[4:8]]
                
                pedestrian['gt_dim'] = [float(i) for i in line[8:11]]
                
                pedestrian['gt_pos'] = [float(i) for i in line[11:14]]
                
                pedestrian['abs_orient'] = float(line[14])*180/math.pi + 180 + 270
                
                pedestrian['abs_orient'] = (pedestrian['abs_orient'] if pedestrian['abs_orient']<360 else pedestrian['abs_orient']-360)
                
                pedestrian['r_angle'] = math.atan2(pedestrian['gt_pos'][0] , pedestrian['gt_pos'][2])*180/math.pi
                
                pedestrian['gt_orient'] = pedestrian['abs_orient'] - pedestrian['r_angle']
                
                pedestrian['gt_orient'] = (pedestrian['gt_orient'] if pedestrian['gt_orient']>0 else 360+pedestrian['gt_orient'])
                
                gt.append(pedestrian)
print('we have', len(gt),'ground truth pedestrians')

we have 2280 ground truth pedestrians


In [64]:
preds = []

for i, label in enumerate(labels_names):
    if label[0:6] in val_labels:
        temp=[]
        with open('../../../eval_kitti/build/results/benchmark-kitti/data/'+label) as inputfile:
            for line in inputfile:
                temp.append(line.strip().split(' '))
        for line in temp:
            if line[0]=='Pedestrian':
                
                pedestrian = {}
                
                pedestrian['image_id'] = label[:6]
                
                pedestrian['truncation'] = float(line[1])
                
                pedestrian['occlusion'] = int(line[2])
                
                pedestrian['pred_obs_angle'] = float(line[3])*180/math.pi + 180
                
                #pedestrian['obs_angle'] = (pedestrian['obs_angle'] if pedestrian['obs_angle']<360 else pedestrian['obs_angle']-360)
                
                pedestrian['ppbox'] = [float(i) for i in line[4:8]]
                
                pedestrian['pred_dim'] = [float(i) for i in line[8:11]]
                
                pedestrian['pred_pos'] = [float(i) for i in line[11:14]]
                
                pedestrian['pred_abs_orient'] = float(line[14])*180/math.pi + 180 + 270
                
                pedestrian['pred_abs_orient'] = (pedestrian['pred_abs_orient'] if pedestrian['pred_abs_orient']<360 else pedestrian['pred_abs_orient']-360)
                
                pedestrian['pred_r_angle'] = math.atan2(pedestrian['pred_pos'][0] , pedestrian['pred_pos'][2])*180/math.pi
                
                pedestrian['pred_orient'] = pedestrian['pred_abs_orient'] - pedestrian['pred_r_angle']
                
                pedestrian['pred_orient'] = (pedestrian['pred_orient'] if pedestrian['pred_orient']>0 else 360+pedestrian['pred_orient'])
                
                preds.append(pedestrian)
print('we have', len(preds),'predicted pedestrians')

we have 1867 predicted pedestrians


#### intersection over union

In [22]:
def get_inter_area(r1, r2):
    left = max(r1[0], r2[0])
    right = min(r1[2], r2[2])
    bottom = max(r1[1], r2[1])
    top = min(r1[3], r2[3])
    if (top-bottom)<0 and (right-left)<0:
        return 0
    else:
        return (top-bottom)*(right-left)

def get_union_area(r1, r2):
    return (r1[2]-r1[0])*(r1[3]-r1[1]) + (r2[2]-r2[0])*(r2[3]-r2[1]) - get_inter_area(r1, r2)

def get_inter_matrix(pp_image, gt_image):
    inter_matrix=[]
    for gt in gt_image:
        inter_line=[]
        gt_box= gt['gt_bbox']
        for pifpaf in pp_image:
            pp_box=pifpaf['ppbox']
            inter_line.append(get_inter_area(gt_box,pp_box)/get_union_area(gt_box, pp_box))
        inter_matrix.append(inter_line)
    return np.array(inter_matrix)

In [23]:
def match_gt_pifpaf(inter_matrix):
    max_val=np.max(inter_matrix)
    matches=[]
    while max_val>0.3:
        args_max=np.unravel_index(np.argmax(inter_matrix, axis=None), inter_matrix.shape)
        inter_matrix[args_max[0],:]=0
        inter_matrix[:,args_max[1]]=0
        matches.append(args_max)
        max_val=np.max(inter_matrix)
    return matches

In [68]:
%%time
gt_2=[]
for i, image_id in enumerate(labels_names):
    
    #update_progress(i/len(labels_names))
    gt_ann = list(filter(lambda d: d['image_id']==image_id[:6], gt))
    
    pp_ann = list(filter(lambda d: d['image_id']==image_id[:6], preds))
    #print(i, len(gt_ann), len(pp_ann))
    matrix=get_inter_matrix(pp_ann, gt_ann)
    
    if matrix.size:
        matches = match_gt_pifpaf(matrix)
        
        for (i,j) in matches:
            pedestrian=cp.deepcopy(gt_ann[i])
            pedestrian['orient_error'] = abs(pp_ann[j]['pred_orient']-pedestrian['gt_orient'])
            pedestrian['orient_error'] = pedestrian['orient_error'] if pedestrian['orient_error'] < 180 else 360-pedestrian['orient_error']
            #pedestrian['keypoints']=pp_ann[j]['keypoints']

            gt_2.append(pedestrian)

CPU times: user 4.81 s, sys: 2.74 ms, total: 4.81 s
Wall time: 4.81 s


In [69]:
print(sum([x['orient_error'] for x in gt_2])/len(gt_2))
    

25.909876624192815


In [26]:
gt_2

[]

### TRY TO ISOLATE ORIENTATION

#### match pifpaf with GT

In [19]:
labels_names=os.listdir('../../../pedestrianDepth-baseline/MonoDepth-PyTorch/data/pif_all/')
labels_names.sort()

In [20]:
val_labels=open("../../intermediate-data/kitti-pifpaf/val.txt", "r").read().split('\n')

In [21]:
pifpafbox = []
for i, label in enumerate(labels_names):
    if label[0:6] in val_labels:
        with open('../../../pedestrianDepth-baseline/MonoDepth-PyTorch/data/pif_all/'+label) as inputfile:
            temp = json.load(inputfile)

            if temp != []:
                for person in temp:
                    new = {}
                    new['ppbox'] = [x/2 for x in person['bbox']]

                    keyp = [x/2 for x in person['keypoints']]
                    
                    
                    #confidence = float(np.sort(np.array(keyp[2::3]))[-3])/2
                    confidence = sum(keyp[2::3])/(2*len(keyp[2::3]))
                    
                    #print(confidence, sum(keyp[2::3])/(2*len(keyp[2::3])))
                    
                    new['confidence'] = confidence

                    del keyp[2::3]

                    new['image_id'] = label[0:6]

                    new['keypoints'] = np.array(keyp).reshape(17,2)
                    
                    if new['confidence']>0.2:
                        #new['confidence']=1.0
                        pifpafbox.append(new)

print('we have',len(pifpafbox),'pifpaf pedestrians')

we have 3671 pifpaf pedestrians


#### enlarge bbox

In [24]:
factor=0.1
for person in pifpafbox:
    person['ppbox'][0]=person['ppbox'][0]-factor*(person['ppbox'][2]-person['ppbox'][0])
    person['ppbox'][1]=person['ppbox'][1]-factor*(person['ppbox'][3]-person['ppbox'][1])
    person['ppbox'][2]=person['ppbox'][2]+factor*(person['ppbox'][2]-person['ppbox'][0])
    person['ppbox'][3]=person['ppbox'][3]+factor*(person['ppbox'][3]-person['ppbox'][1])

In [16]:
labels_names=os.listdir('../../../pedestrianDepth-baseline/MonoDepth-PyTorch/data/training/label_2/')
labels_names.sort()

#### reload gt

In [20]:
gt = []

for i, label in enumerate(labels_names):
    if label[0:6] in val_labels:
        temp=[]
        with open('../../../pedestrianDepth-baseline/MonoDepth-PyTorch/data/training/label_2/'+label) as inputfile:
            for line in inputfile:
                temp.append(line.strip().split(' '))
        for line in temp:
            if line[0]=='Pedestrian':
                
                pedestrian = {}
                
                pedestrian['image_id'] = label[:6]
                
                pedestrian['truncation'] = float(line[1])
                
                pedestrian['occlusion'] = int(line[2])
                
                pedestrian['obs_angle'] = float(line[3])*180/math.pi + 180
                
                #pedestrian['obs_angle'] = (pedestrian['obs_angle'] if pedestrian['obs_angle']<360 else pedestrian['obs_angle']-360)
                
                pedestrian['gt_bbox'] = [float(i) for i in line[4:8]]
                
                pedestrian['gt_dim'] = [float(i) for i in line[8:11]]
                
                pedestrian['gt_pos'] = [float(i) for i in line[11:14]]
                
                pedestrian['abs_orient'] = float(line[14])*180/math.pi + 180 + 270
                
                pedestrian['abs_orient'] = (pedestrian['abs_orient'] if pedestrian['abs_orient']<360 else pedestrian['abs_orient']-360)
                
                pedestrian['r_angle'] = math.atan2(pedestrian['gt_pos'][0] , pedestrian['gt_pos'][2])*180/math.pi
                
                pedestrian['gt_orient'] = pedestrian['abs_orient'] - pedestrian['r_angle']
                
                pedestrian['gt_orient'] = (pedestrian['gt_orient'] if pedestrian['gt_orient']>0 else 360+pedestrian['gt_orient'])
                
                gt.append(pedestrian)
print('we have', len(gt),'ground truth pedestrians')

we have 2280 ground truth pedestrians


In [25]:
%%time
matched_pedestrians = []
for i, image_id in enumerate(labels_names):
    
    #update_progress(i/len(labels_names))
    gt_ann = list(filter(lambda d: d['image_id']==image_id[:6], gt))
    
    pp_ann = list(filter(lambda d: d['image_id']==image_id[:6], pifpafbox))
    #print(i, len(gt_ann), len(pp_ann))
    matrix=get_inter_matrix(pp_ann, gt_ann)
    
    if matrix.size:
        matches = match_gt_pifpaf(matrix)
        
        for (i,j) in matches:
            pedestrian=cp.deepcopy(gt_ann[i])
            #pedestrian['orient_error'] = abs(pp_ann[j]['pred_orient']-pedestrian['gt_orient'])
            #pedestrian['orient_error'] = pedestrian['orient_error'] if pedestrian['orient_error'] < 180 else 360-pedestrian['orient_error']
            pedestrian['keypoints']=pp_ann[j]['keypoints']
            pedestrian['confidence']=pp_ann[j]['confidence']
            pedestrian['ppbox']=pp_ann[j]['ppbox']
            matched_pedestrians.append(pedestrian)
            
            
            

CPU times: user 5.14 s, sys: 0 ns, total: 5.14 s
Wall time: 5.15 s


In [26]:
len(matched_pedestrians)

1375

In [27]:
matched_pedestrians[0]

{'image_id': '000005',
 'truncation': 0.0,
 'occlusion': 0,
 'obs_angle': 291.1538122553797,
 'gt_bbox': [330.06, 178.74, 360.77, 238.64],
 'gt_dim': [1.87, 0.96, 0.65],
 'gt_pos': [-8.5, 2.07, 23.02],
 'abs_orient': 181.10028942580084,
 'r_angle': -20.26637142000956,
 'gt_orient': 201.3666608458104,
 'keypoints': array([[343.95, 186.7 ],
        [344.8 , 185.65],
        [343.05, 185.6 ],
        [346.45, 185.85],
        [341.15, 186.05],
        [349.7 , 191.85],
        [338.45, 191.45],
        [352.5 , 200.95],
        [333.9 , 199.35],
        [352.95, 209.65],
        [331.05, 207.15],
        [347.25, 209.6 ],
        [339.9 , 209.45],
        [345.7 , 220.85],
        [341.  , 221.5 ],
        [344.9 , 235.  ],
        [347.15, 232.55]]),
 'confidence': 0.835294117647059,
 'ppbox': [326.2231389166713,
  174.68167937994002,
  358.28899074296356,
  247.0032739150524]}

In [28]:
inputs = np.array([x['keypoints'].flatten() for x in matched_pedestrians])
with torch.no_grad():
    net = mlmodel.LinearModel(34, 8)
    net=net.to(device)
    net.eval()
    params = torch.load('../../intermediate-data/kitti-pifpaf/net-kitti-multi.pkl')
    net.load_state_dict(params)
    out_sample = infer(inputs , net)

In [29]:
for i, detection in enumerate(matched_pedestrians):
    detection['pred_orient'] = np.round(out_sample[0][i],2)
    detection['pred_dim'] = np.round(out_sample[1][i],2)
    detection['pred_pos'] = np.round(out_sample[2][i],2)
    detection['obs_angle'] = math.atan2(detection['pred_pos'][2],detection['pred_pos'][0])*180/math.pi
    detection['truncation'] = -1.0
    detection['occlusion'] = -1

In [30]:
for label in val_labels:
    #print(label)
    pedestrians = [d for d in matched_pedestrians if d['image_id'] in [label]]
    
    text_file = open('../../../eval_kitti/build/results/benchmark-kitti/data/'+label+".txt", "w")
    
    for pedestrian in pedestrians:
        obs_angle = (pedestrian['obs_angle']-180)*math.pi/180
        
        bbox = [min(pedestrian['keypoints'][:,0]), min(pedestrian['keypoints'][:,1]),
                max(pedestrian['keypoints'][:,0]), max(pedestrian['keypoints'][:,1])]
        
        r_angle = math.atan2(pedestrian['pred_pos'][0] , pedestrian['pred_pos'][2])*180/math.pi
        
        abs_orient = pedestrian['pred_orient']+r_angle
        
        abs_orient -= 90
        
        #abs_orient = (abs_orient if abs_orient<360 else abs_orient-360)
        
        if abs_orient>180:
            abs_orient = abs_orient-360
        if abs_orient<-180:
            abs_orient = abs_orient+360
            
        
        abs_orient = abs_orient*math.pi/180
        
        
        
        
        #print(pedestrian)
        to_write = ("Pedestrian "+str(pedestrian['truncation'])+" "+str(pedestrian['occlusion'])+" "+str(round(obs_angle,2))+" "+str(pedestrian['gt_bbox'][0])
                    +' '+str(pedestrian['gt_bbox'][1])+' '+str(pedestrian['gt_bbox'][2])+' '+str(pedestrian['gt_bbox'][3])+' '+str(round(pedestrian['gt_dim'][0],2))
                    +' '+str(round(pedestrian['gt_dim'][1],2))+' '
                    +str(round(pedestrian['gt_dim'][2],2))+' '+str(round(pedestrian['gt_pos'][0],2))+' '+
                    str(round(pedestrian['gt_pos'][1],2))+' '+str(round(pedestrian['gt_pos'][2],2))+' '+
                    str(round(abs_orient,2))+' '+str(round(pedestrian['confidence'],2))+'\n')
        text_file.write(to_write)
    text_file.close()

### LOAD M3D

In [31]:
m3d_preds = []

for i, label in enumerate(labels_names):
    if label[0:6] in val_labels:
        temp=[]
        with open('../../../eval_kitti/build/results/m3d/data/'+label) as inputfile:
            for line in inputfile:
                #print(line)
                temp.append(line.strip().split(' '))
        for line in temp:
            if line[0]=='pedestrian':
                
                pedestrian = {}
                
                pedestrian['image_id'] = label[:6]
                
                pedestrian['truncation'] = float(line[1])
                
                pedestrian['occlusion'] = int(line[2])
                
                pedestrian['pred_obs_angle'] = float(line[3])*180/math.pi + 180
                
                #pedestrian['obs_angle'] = (pedestrian['obs_angle'] if pedestrian['obs_angle']<360 else pedestrian['obs_angle']-360)
                
                pedestrian['ppbox'] = [float(i) for i in line[4:8]]
                
                pedestrian['pred_dim'] = [float(i) for i in line[8:11]]
                
                pedestrian['pred_pos'] = [float(i) for i in line[11:14]]
                
                pedestrian['m3d_confidence'] = float(line[15])
                
                pedestrian['pred_abs_orient'] = float(line[14])*180/math.pi + 180 + 270
                
                pedestrian['pred_abs_orient'] = (pedestrian['pred_abs_orient'] if pedestrian['pred_abs_orient']<360 else pedestrian['pred_abs_orient']-360)
                
                pedestrian['pred_r_angle'] = math.atan2(pedestrian['pred_pos'][0] , pedestrian['pred_pos'][2])*180/math.pi
                
                pedestrian['pred_orient'] = pedestrian['pred_abs_orient'] - pedestrian['pred_r_angle']
                
                pedestrian['pred_orient'] = (pedestrian['pred_orient'] if pedestrian['pred_orient']>0 else 360+pedestrian['pred_orient'])
                
                m3d_preds.append(pedestrian)
print('we have', len(m3d_preds),'predicted pedestrians')

we have 4077 predicted pedestrians


In [32]:
m3d_preds[2]

{'image_id': '000005',
 'truncation': -1.0,
 'occlusion': -1,
 'pred_obs_angle': 243.59831525952137,
 'ppbox': [329.0, 180.0, 358.54, 239.45],
 'pred_dim': [1.76, 0.66, 0.49],
 'pred_pos': [-7.7, 1.77, 20.8],
 'm3d_confidence': 1.0,
 'pred_abs_orient': 132.97183463481173,
 'pred_r_angle': -20.314164737736107,
 'pred_orient': 153.28599937254785}

In [33]:
%%time
commun_pedestrians = []
for i, image_id in enumerate(labels_names):
    
    #update_progress(i/len(labels_names))
    gt_ann = list(filter(lambda d: d['image_id']==image_id[:6], matched_pedestrians))
    
    pp_ann = list(filter(lambda d: d['image_id']==image_id[:6], m3d_preds))
    #print(i, len(gt_ann), len(pp_ann))
    matrix=get_inter_matrix(pp_ann, gt_ann)
    
    if matrix.size:
        matches = match_gt_pifpaf(matrix)
        
        for (i,j) in matches:
            pedestrian=cp.deepcopy(gt_ann[i])
            #pedestrian['orient_error'] = abs(pp_ann[j]['pred_orient']-pedestrian['gt_orient'])
            #pedestrian['orient_error'] = pedestrian['orient_error'] if pedestrian['orient_error'] < 180 else 360-pedestrian['orient_error']
            #pedestrian['keypoints']=pp_ann[j]['keypoints']
            pedestrian['m3d_confidence']=pp_ann[j]['m3d_confidence']
            pedestrian['m3d_orient']=pp_ann[j]['pred_orient']
            #pedestrian['ppbox']=pp_ann[j]['ppbox']
            commun_pedestrians.append(pedestrian)
            

CPU times: user 7.46 s, sys: 0 ns, total: 7.46 s
Wall time: 7.61 s


In [34]:
len(commun_pedestrians)

1310

In [87]:
commun_pedestrians[0]

{'image_id': '000005',
 'truncation': 0.0,
 'occlusion': 0,
 'obs_angle': 110.17837983380915,
 'gt_bbox': [330.06, 178.74, 360.77, 238.64],
 'gt_dim': [1.87, 0.96, 0.65],
 'gt_pos': [-8.5, 2.07, 23.02],
 'abs_orient': 181.10028942580084,
 'r_angle': -20.26637142000956,
 'gt_orient': 201.3666608458104,
 'keypoints': array([[343.95, 186.7 ],
        [344.8 , 185.65],
        [343.05, 185.6 ],
        [346.45, 185.85],
        [341.15, 186.05],
        [349.7 , 191.85],
        [338.45, 191.45],
        [352.5 , 200.95],
        [333.9 , 199.35],
        [352.95, 209.65],
        [331.05, 207.15],
        [347.25, 209.6 ],
        [339.9 , 209.45],
        [345.7 , 220.85],
        [341.  , 221.5 ],
        [344.9 , 235.  ],
        [347.15, 232.55]]),
 'confidence': 0.835294117647059,
 'ppbox': [328.8732093155384,
  180.65867066383362,
  355.37391330420974,
  240.42858350276947],
 'pred_orient': 194.2,
 'pred_dim': array([1.57, 0.57, 0.47], dtype=float32),
 'pred_pos': array([-7.35,  1.7

In [35]:
for label in val_labels:
    #print(label)
    pedestrians = [d for d in commun_pedestrians if d['image_id'] in [label]]
    
    text_file = open('../../../eval_kitti/build/results/benchmark-kitti/data/'+label+".txt", "w")
    
    for pedestrian in pedestrians:
        obs_angle = (pedestrian['obs_angle']-180)*math.pi/180
        
        bbox = [min(pedestrian['keypoints'][:,0]), min(pedestrian['keypoints'][:,1]),
                max(pedestrian['keypoints'][:,0]), max(pedestrian['keypoints'][:,1])]
        
        r_angle = math.atan2(pedestrian['gt_pos'][0] , pedestrian['gt_pos'][2])*180/math.pi
        
        gt_orient = pedestrian['gt_orient']+r_angle
        
        abs_orient = pedestrian['pred_orient']+r_angle
        
        abs_orient -= 90
        
        gt_orient -= 90
        
        #abs_orient = (abs_orient if abs_orient<360 else abs_orient-360)
        
        if abs_orient>180:
            abs_orient = abs_orient-360
            gt_orient = gt_orient-360
        if gt_orient<-180:
            abs_orient = abs_orient+360
            gt_orient = gt_orient+360
            
        
        abs_orient = abs_orient*math.pi/180
        gt_orient = gt_orient*math.pi/180
        print(round(gt_orient,2),round(abs_orient,2))
        
        #print(pedestrian)
        to_write = ("Pedestrian "+str(pedestrian['truncation'])+" "+str(pedestrian['occlusion'])+" "+str(round(obs_angle,2))+" "+str(pedestrian['gt_bbox'][0])
                    +' '+str(pedestrian['gt_bbox'][1])+' '+str(pedestrian['gt_bbox'][2])+' '+str(pedestrian['gt_bbox'][3])+' '+str(round(pedestrian['gt_dim'][0],2))
                    +' '+str(round(pedestrian['gt_dim'][1],2))+' '
                    +str(round(pedestrian['gt_dim'][2],2))+' '+str(round(pedestrian['gt_pos'][0],2))+' '+
                    str(round(pedestrian['gt_pos'][1],2))+' '+str(round(pedestrian['gt_pos'][2],2))+' '+
                    str(round(abs_orient,2))+' '+str(round(pedestrian['confidence'],2))+'\n')
        text_file.write(to_write)
    text_file.close()

000001
000002
000004
000005
1.59 1.46
000006
000008
000015
1.25 0.93
4.8 -1.04
000019
000020
000021
000023
000024
000025
000027
000028
-2.97 -2.84
000031
000033
000035
000037
000039
000040
1.24 1.24
000042
000047
000048
-1.55 -1.66
4.79 -1.32
-1.61 -2.28
000050
000052
000053
000058
3.14 3.28
000059
000061
000062
000063
000065
2.67 2.55
000066
000076
-1.55 -1.33
-0.88 5.15
1.58 0.79
1.6 1.07
1.9 1.14
000077
000078
000081
000089
000090
000093
000094
000098
000102
000104
000106
000107
000108
000116
000117
000118
2.71 3.17
000122
000124
000126
000128
000132
000134
0.1 0.35
2.8 3.41
3.12 3.05
000135
000137
000139
000140
000143
000147
-1.76 -1.83
000151
000152
000153
1.99 1.29
2.1 1.42
000156
2.91 3.04
000159
000161
000167
-1.6 -1.51
000168
000169
000170
000173
000174
000175
000181
000182
000183
000186
1.1 0.77
000187
1.59 1.1
000188
000190
000191
000192
000194
000195
1.53 1.67
000196
000197
000199
-3.13 -3.07
000201
000203
000204
000207
000211
000212
000213
000216
000218
000223
000224
00022

1.26 0.75
1.35 1.1
1.43 1.44
1.28 1.32
1.38 1.23
0.59 0.86
1.29 1.96
0.87 1.14
1.24 1.11
1.12 1.21
3.28 2.57
1.4 0.97
1.32 1.28
1.0 0.85
002709
4.73 -0.91
002710
002711
002712
002713
002715
1.55 2.28
002717
002720
002721
0.88 1.01
002722
002724
002725
1.2 1.11
-2.95 -3.14
1.56 1.47
002726
002727
002728
002729
002730
002735
3.16 3.07
002737
002740
002742
1.3 1.16
1.1 1.24
1.35 1.28
1.15 1.18
1.21 1.18
1.1 1.27
1.29 1.24
1.16 1.22
0.96 1.27
002744
002745
002746
002747
002748
002749
002752
002753
002755
002757
002758
002760
002761
002763
-3.06 -3.01
002764
002765
002767
1.6 1.47
002772
002773
002775
1.54 2.07
-1.47 -0.92
4.73 -0.75
2.3 1.64
4.77 -1.26
1.58 0.19
-1.53 -0.98
002783
002786
002787
002789
002793
002794
002796
002797
002800
1.33 1.1
-1.66 -1.89
002801
002804
002805
002806
002809
002810
002811
1.57 1.66
002812
002814
002815
002818
002820
002826
002827
002828
002830
002831
002833
002836
002839
002840
002841
002844
002845
002846
002847
002848
002853
002856
002858
002861
002863
1.5

-1.57 -1.54
004999
005001
005002
005004
005008
005010
005013
005014
005015
005017
-3.08 -2.99
005019
005021
005024
005026
005028
005032
005034
005036
005037
1.82 1.52
1.89 1.87
005038
005040
0.97 1.26
005041
005045
005049
005050
0.0 0.1
005052
005053
005054
005055
005056
005057
005058
005062
005063
005064
-0.02 0.46
005065
1.42 1.13
-1.57 4.33
005067
005068
005070
005072
005073
005074
005075
005077
-1.57 -1.48
005078
1.64 1.23
1.79 1.34
005079
005080
005081
005082
-1.43 -1.59
2.58 0.86
1.69 1.64
1.46 0.07
4.74 -1.28
1.58 1.65
-1.52 -1.57
4.73 -0.64
005086
-2.18 -3.04
-1.6 -1.42
4.48 -1.08
-1.53 -1.41
-2.95 -2.62
2.3 1.96
-2.45 -1.58
-2.85 -2.83
005090
005093
005094
005095
005101
005103
005105
005108
005109
005110
005112
005113
005120
005121
005122
005124
-3.12 -3.11
005125
005127
005128
005133
2.57 2.14
3.13 4.16
005135
005136
005138
005139
005140
005141
005143
005144
005145
005147
1.53 1.99
005149
1.32 1.44
005153
005155
005156
005157
005158
005161
005162
005163
005164
005166
005167
0

-3.03 -2.79
007112
007115
007117
007119
007120
007122
007125
007130
007131
007132
007133
007135
1.53 1.33
007136
4.51 -1.93
007138
007139
007144
007145
007146
007149
3.11 3.33
007154
007157
007158
007161
1.37 1.06
4.66 -0.7
007162
007163
007164
007165
007166
007168
007169
007172
007174
007176
007177
007178
3.12 3.7
007180
007182
007183
007187
007194
007198
007199
1.53 1.38
007200
007201
2.63 1.67
007202
1.39 0.45
1.57 1.3
-2.27 -2.07
-1.56 4.48
-2.83 -2.71
-0.95 -0.79
1.54 1.89
1.36 0.52
007204
007205
007207
007208
007210
007212
007214
007215
007217
007219
007221
007225
007227
007229
007230
007232
007233
007235
007238
007240
007242
007244
007246
007247
007252
007253
1.59 1.43
007255
007256
007258
007260
007261
007262
007265
-1.46 -1.46
1.64 0.9
1.78 1.55
-1.54 -0.39
-1.39 -0.88
007266
007267
007271
007272
-0.02 0.05
007273
007274
007275
007277
007278
3.13 3.2
007279
007280
007283
007284
007287
007288
-0.05 0.04
007289
007290
1.85 2.1
2.04 1.35
007291
007292
007294
007299
007300
007302


In [36]:
for label in val_labels:
    #print(label)
    pedestrians = [d for d in commun_pedestrians if d['image_id'] in [label]]
    
    text_file = open('../../../eval_kitti/build/results/m3d_matched/data/'+label+".txt", "w")
    
    for pedestrian in pedestrians:
        obs_angle = (pedestrian['obs_angle']-180)*math.pi/180
        
        bbox = [min(pedestrian['keypoints'][:,0]), min(pedestrian['keypoints'][:,1]),
                max(pedestrian['keypoints'][:,0]), max(pedestrian['keypoints'][:,1])]
        
        r_angle = math.atan2(pedestrian['gt_pos'][0] , pedestrian['gt_pos'][2])*180/math.pi
        
        abs_orient = pedestrian['m3d_orient']+r_angle
        
        abs_orient -= 90
        
        #abs_orient = (abs_orient if abs_orient<360 else abs_orient-360)
        
        if abs_orient>180:
            abs_orient = abs_orient-360
        if abs_orient<-180:
            abs_orient = abs_orient+360
            
        
        abs_orient = abs_orient*math.pi/180
        
        
        #print(round(abs_orient,2))
        
        #print(pedestrian)
        to_write = ("Pedestrian "+str(pedestrian['truncation'])+" "+str(pedestrian['occlusion'])+" "+str(round(obs_angle,2))+" "+str(pedestrian['gt_bbox'][0])
                    +' '+str(pedestrian['gt_bbox'][1])+' '+str(pedestrian['gt_bbox'][2])+' '+str(pedestrian['gt_bbox'][3])+' '+str(round(pedestrian['gt_dim'][0],2))
                    +' '+str(round(pedestrian['gt_dim'][1],2))+' '
                    +str(round(pedestrian['gt_dim'][2],2))+' '+str(round(pedestrian['gt_pos'][0],2))+' '+
                    str(round(pedestrian['gt_pos'][1],2))+' '+str(round(pedestrian['gt_pos'][2],2))+' '+
                    str(round(abs_orient,2))+' '+str(round(pedestrian['m3d_confidence'],2))+'\n')
        text_file.write(to_write)
        
    text_file.close()

In [37]:
for label in val_labels:
    print(label)
    pedestrians = [d for d in commun_pedestrians if d['image_id'] in [label]]
    
    text_file = open('../../../eval_kitti/build/results/ground-truth/data/'+label+".txt", "w")
    
    for pedestrian in pedestrians:
        obs_angle = (pedestrian['obs_angle']-180)*math.pi/180
        
        bbox = [min(pedestrian['keypoints'][:,0]), min(pedestrian['keypoints'][:,1]),
                max(pedestrian['keypoints'][:,0]), max(pedestrian['keypoints'][:,1])]
        
        r_angle = math.atan2(pedestrian['gt_pos'][0] , pedestrian['gt_pos'][2])*180/math.pi
        
        abs_orient = pedestrian['gt_orient']+r_angle
        
        abs_orient -= 90
        
        #abs_orient = (abs_orient if abs_orient<360 else abs_orient-360)
        
        if abs_orient>180:
            abs_orient = abs_orient-360
        if abs_orient<-180:
            abs_orient = abs_orient+360
            
        
        abs_orient = abs_orient*math.pi/180
        
        
        print(round(abs_orient,2))
        
        #print(pedestrian)
        to_write = ("Pedestrian "+str(pedestrian['truncation'])+" "+str(pedestrian['occlusion'])+" "+str(round(obs_angle,2))+" "+str(pedestrian['gt_bbox'][0])
                    +' '+str(pedestrian['gt_bbox'][1])+' '+str(pedestrian['gt_bbox'][2])+' '+str(pedestrian['gt_bbox'][3])+' '+str(round(pedestrian['gt_dim'][0],2))
                    +' '+str(round(pedestrian['gt_dim'][1],2))+' '
                    +str(round(pedestrian['gt_dim'][2],2))+' '+str(round(pedestrian['gt_pos'][0],2))+' '+
                    str(round(pedestrian['gt_pos'][1],2))+' '+str(round(pedestrian['gt_pos'][2],2))+' '+
                    str(round(abs_orient,2))+' '+str(round(1.00,2))+'\n')
        text_file.write(to_write)
        
    text_file.close()

000001
000002
000004
000005
1.59
000006
000008
000015
1.25
-1.48
000019
000020
000021
000023
000024
000025
000027
000028
-2.97
000031
000033
000035
000037
000039
000040
1.24
000042
000047
000048
-1.55
-1.49
-1.61
000050
000052
000053
000058
3.14
000059
000061
000062
000063
000065
2.67
000066
000076
-1.55
-0.88
1.58
1.6
1.9
000077
000078
000081
000089
000090
000093
000094
000098
000102
000104
000106
000107
000108
000116
000117
000118
2.71
000122
000124
000126
000128
000132
000134
0.1
2.8
3.12
000135
000137
000139
000140
000143
000147
-1.76
000151
000152
000153
1.99
2.1
000156
2.91
000159
000161
000167
-1.6
000168
000169
000170
000173
000174
000175
000181
000182
000183
000186
1.1
000187
1.59
000188
000190
000191
000192
000194
000195
1.53
000196
000197
000199
-3.13
000201
000203
000204
000207
000211
000212
000213
000216
000218
000223
000224
000226
000229
000230
000231
000234
000235
000236
000237
000239
000242
000246
000247
000248
000249
000250
000251
-3.11
-0.26
000252
000260
000262
00026

002378
002380
002382
002383
002384
0.98
002385
002386
1.57
-1.62
0.34
-1.44
1.59
3.08
002387
002391
002392
002393
1.65
2.42
-2.34
-1.43
002397
002398
002399
002404
002405
002411
002414
002415
002418
002419
002420
002422
002423
002424
-0.85
002425
002428
002429
002432
002433
002434
002439
002440
1.54
1.56
-1.5
-0.95
-1.99
1.35
-1.52
-1.57
1.42
002442
002446
002450
002454
002455
002457
1.59
002458
002460
002461
002462
002463
002473
002474
002476
002477
-2.02
1.63
2.35
-2.1
-1.6
-2.95
-2.43
-1.69
1.53
002478
-0.11
002479
002483
002486
002488
002490
002492
002495
002497
002499
002500
002502
002503
002504
002505
1.56
002506
002509
002511
002516
002519
002520
002521
002525
002526
1.55
-1.06
1.63
-1.53
-1.64
002528
002529
1.57
002530
002531
002532
002534
002538
002539
002540
002541
002543
002546
002548
2.64
002552
002556
002557
-1.6
002558
002562
002563
002564
1.36
002565
0.01
0.42
002568
002569
002570
002572
002574
002575
002577
002580
002581
002583
-1.53
-2.52
-1.58
-1.64
002584
002585
0025

004640
004644
004647
004648
004649
004650
004651
-2.2
004652
004655
004657
-1.53
004658
004660
004665
-1.64
-1.56
-1.59
004666
004667
004668
004669
004672
004673
004679
-3.14
004680
004682
004683
004685
004686
004687
004688
004689
004691
004692
004693
004694
004695
004697
004698
004699
004700
004705
004706
004708
004709
004710
004711
004713
004714
004715
004716
004717
-2.2
004718
004720
0.06
0.0
004721
004722
2.66
004724
3.01
004725
004726
004730
004732
004734
004735
004737
004738
004739
004740
1.58
2.29
-1.43
-1.4
1.48
-1.53
-1.52
1.6
004742
004743
004744
004745
004746
004748
004752
004753
004756
004759
004762
004763
004764
004766
004768
004769
004770
004773
004776
004777
004782
004783
004787
004788
0.0
0.34
004790
0.98
004791
004792
004797
0.0
004799
004800
004804
0.01
0.66
3.12
0.15
004806
004807
004810
004811
004813
-2.22
004814
004815
004816
004817
-2.24
004821
004822
004825
1.87
0.04
1.47
-1.57
004829
004830
004831
004832
004835
004839
004843
004846
004848
004849
004850
2.57
-3.1

0.0
007449
007450
007453
007456
007458
007462
007463
007464
1.31
007466
007467
007468
007469
-1.57
007470
007473
007475
007477
-3.13
007478
007480


In [40]:
for label in val_labels:
    print(label)
    pedestrians = [d for d in gt if d['image_id'] in [label]]
    
    text_file = open('../../../eval_kitti/build/results/ground-truth/data/'+label+".txt", "w")
    
    for pedestrian in pedestrians:
        obs_angle = (pedestrian['obs_angle']-180)*math.pi/180
        
        #bbox = [min(pedestrian['keypoints'][:,0]), min(pedestrian['keypoints'][:,1]),
         #       max(pedestrian['keypoints'][:,0]), max(pedestrian['keypoints'][:,1])]
        
        r_angle = math.atan2(pedestrian['gt_pos'][0] , pedestrian['gt_pos'][2])*180/math.pi
        
        abs_orient = pedestrian['gt_orient']+r_angle
        
        abs_orient -= 90
        
        #abs_orient = (abs_orient if abs_orient<360 else abs_orient-360)
        
        if abs_orient>180:
            abs_orient = abs_orient-360
        if abs_orient<-180:
            abs_orient = abs_orient+360
            
        
        abs_orient = abs_orient*math.pi/180
        
        
        print(round(abs_orient,2))
        
        #print(pedestrian)
        to_write = ("Pedestrian "+str(pedestrian['truncation'])+" "+str(pedestrian['occlusion'])+" "+str(round(obs_angle,2))+" "+str(pedestrian['gt_bbox'][0])
                    +' '+str(pedestrian['gt_bbox'][1])+' '+str(pedestrian['gt_bbox'][2])+' '+str(pedestrian['gt_bbox'][3])+' '+str(round(pedestrian['gt_dim'][0],2))
                    +' '+str(round(pedestrian['gt_dim'][1],2))+' '
                    +str(round(pedestrian['gt_dim'][2],2))+' '+str(round(pedestrian['gt_pos'][0],2))+' '+
                    str(round(pedestrian['gt_pos'][1],2))+' '+str(round(pedestrian['gt_pos'][2],2))+' '+
                    str(round(abs_orient,2))+' '+str(round(1.00,2))+'\n')
        text_file.write(to_write)
        
    text_file.close()

000001
000002
000004
000005
1.59
000006
000008
000015
1.25
-1.48
-1.46
-1.54
000019
000020
000021
000023
000024
000025
000027
000028
-2.97
000031
000033
000035
-0.03
000037
000039
000040
1.24
000042
000047
000048
-1.49
-1.55
-1.61
000050
000052
000053
000058
3.14
000059
000061
000062
000063
-2.18
000065
2.67
000066
000076
1.9
-0.88
0.22
-1.61
-1.55
2.53
1.6
1.58
000077
000078
000081
000089
000090
000093
000094
000098
000102
000104
000106
000107
000108
000116
000117
000118
2.71
000122
000124
3.14
0.07
000126
000128
000132
000134
0.1
0.0
0.15
0.13
3.12
2.8
-3.13
000135
000137
000139
000140
000143
000147
-1.76
000151
-0.22
000152
000153
1.84
2.1
1.99
000156
2.91
000159
000161
000167
-1.6
000168
000169
000170
000173
000174
000175
000181
000182
000183
000186
1.1
000187
1.59
000188
000190
-1.61
-1.54
000191
000192
000194
1.33
000195
1.53
000196
000197
000199
-3.13
000201
1.35
1.41
000203
000204
000207
1.55
000211
000212
000213
000216
000218
000223
000224
000226
000229
000230
000231
000234
00

001660
001662
-2.24
001664
001665
-2.22
001666
001667
2.49
001670
001675
001680
001682
-1.36
-1.4
001683
001684
001689
3.07
001693
-1.66
-1.65
-2.74
-1.82
-1.93
-2.9
2.35
1.61
1.53
-2.36
001694
001697
001699
001701
001702
1.51
001704
001705
001706
-1.57
001707
001709
001710
001711
001712
2.56
001713
-1.54
-1.48
001714
0.52
0.0
0.15
0.07
3.12
2.75
-3.12
001717
001718
001719
001721
001722
001726
001727
001729
001732
001733
001740
001741
001742
001745
001746
001749
001750
1.55
001751
001752
001755
001758
-3.07
001762
001764
001765
001768
001771
001772
001774
001776
2.84
2.8
2.8
1.57
-3.06
-2.29
1.62
001778
001780
001781
001782
-0.14
001783
001786
-1.57
001787
001794
001795
-1.55
-1.55
001797
001800
001801
001802
001804
1.57
001807
001808
001813
001814
001817
-0.02
001818
001820
001822
-1.49
1.75
1.61
-1.52
-1.41
001823
001824
001825
001828
001831
001835
3.12
001840
001844
001846
0.0
001848
3.13
001851
001852
001853
3.13
001854
001855
001856
001858
001859
001861
001862
001863
001867
001868

004126
004128
004129
004130
004131
004132
004136
1.57
004137
004138
004140
004142
1.3
1.42
1.39
-1.9
-1.95
1.34
1.16
-1.94
-1.53
0.79
-0.09
-0.9
004143
004148
004149
-1.64
-1.61
004150
004152
004153
004154
004155
004156
004157
004158
004160
004161
004162
004163
004164
004168
004171
004172
004173
-1.57
004174
004175
004185
004187
004188
004189
004190
004191
004195
004196
004202
004205
004206
004207
004209
004210
004213
004214
004215
-2.95
3.13
-0.12
2.81
004220
004221
004222
-1.57
004223
2.92
004224
004226
0.99
004228
004232
004237
0.0
004239
004241
004242
004243
004246
004248
004249
004250
1.76
1.61
1.55
004251
-2.22
004254
004255
004256
004259
004260
004263
004270
004271
3.0
2.8
-0.14
004275
004277
004278
004280
004281
1.0
004282
004284
004285
004288
004289
004290
-1.57
-1.57
-1.57
004291
1.56
004293
004294
004295
004298
004299
004300
004301
-1.54
-1.56
-1.57
004303
004305
1.59
004306
004307
004309
004311
004312
004314
004318
004319
004321
-1.48
-1.57
-3.0
2.57
2.94
-2.81
-2.15
004323

1.37
006125
006126
006127
006130
-1.38
-0.95
2.86
1.62
1.54
-1.57
-1.68
006133
006136
006139
-2.65
0.51
1.45
1.43
1.49
1.19
1.1
1.14
-2.86
1.29
1.21
1.37
1.34
1.33
1.24
1.34
1.32
1.37
006144
006146
006148
3.14
0.07
006151
-3.13
006152
0.5
006154
006156
006161
006163
006165
006167
006168
006169
006173
006176
-1.62
-1.62
-1.68
-1.32
006177
006182
006185
1.35
006186
006187
006190
-1.57
-1.58
006194
-3.11
006195
006196
006198
-1.52
-1.54
-1.57
006202
006204
006208
006210
3.04
2.77
-0.2
006213
-0.05
2.83
006215
3.1
006219
006222
0.0
006227
006228
-1.53
006229
-1.57
-1.58
006232
1.49
006233
006238
006240
-0.1
006244
006246
006247
006249
006250
006258
0.2
006263
006265
006266
006267
006269
006270
006272
006273
006274
1.27
006275
006276
0.22
0.0
3.02
006278
006280
006282
006286
006287
006288
006297
006300
006301
3.12
006302
006305
006306
006312
006314
006315
006316
006317
006321
006322
006324
006331
006332
006333
006334
-1.57
-1.57
006338
006339
006340
006342
006343
006344
006345
006348
006349